In [85]:
import numpy as np
import pandas as pd
import plotly
import plotly.express as px
plotly.offline.init_notebook_mode (connected = True)
import matplotlib.pyplot as plt
import warnings

In [91]:
header = 0 # 0 for No, 1 for Yes
label = 1 # 0 for No, 1 for Yes
path="iris.data.csv" #path to the datset
visualization=1 # 0 for No, 1 for Yes
apply = 1 # 0 for none, 1 for standardization, 2 for normalization

In [104]:
if header==1:
    df = pd.read_csv(path)
else:
    df = pd.read_csv(path,header=None)

if label==1:
    labeled=df[df.columns[-1]]
    df=df.drop([df.columns[-1]],axis=1)

In [76]:
df.head()

,0,1,2,3
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [77]:
#warnings.filterwarnings("ignore")

In [81]:
if visualization==1:
    df.corr().style.background_gradient(cmap='coolwarm')

In [79]:
feats = df.columns[:]
feats

Int64Index([0, 1, 2, 3], dtype='int64')

In [98]:
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

if apply==1:
    scaler = StandardScaler().fit_transform(df[feats])
elif apply==2:
    scaler = preprocessing.normalize(df[feats])
else:
    scaler = df[feats]

In [99]:
scaled = pd.DataFrame(scaler, columns=feats)
scaled

,0,1,2,3
0,0.803773,0.551609,0.220644,0.031521
1,0.828133,0.507020,0.236609,0.033801
2,0.805333,0.548312,0.222752,0.034269
3,0.800030,0.539151,0.260879,0.034784
4,0.790965,0.569495,0.221470,0.031639
...,...,...,...,...
145,0.721557,0.323085,0.560015,0.247699
146,0.729654,0.289545,0.579090,0.220054
147,0.716539,0.330710,0.573231,0.220474
148,0.674671,0.369981,0.587616,0.250281


In [101]:
import plotly.express as px

if visualization==1:
    features = feats

    fig = px.scatter_matrix(
        scaled,
        dimensions=features,
        color="STATUS"
    )
    fig.update_traces(diagonal_visible=True)
    fig.show()